In [1]:
# for mxnet
! pip install --upgrade mxnet
!  pip install facenet-pytorch
# ! pip install tf2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.4 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.functional as F
import matplotlib.pyplot as plt
import numpy as np
import mxnet as mx
from mxnet import gluon
from mxnet.gluon.data.vision import transforms
from facenet_pytorch import InceptionResnetV1
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using cuda device


In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

dataset = gluon.data.vision.ImageRecordDataset('/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset_webcasio/train.rec' )


In [ ]:
images = np.array([dataset[0][0].asnumpy()])
labels = [dataset[0][1]]

In [ ]:
for item in range(1 , 6000):
  images = np.concatenate( (images , np.array([dataset[item][0].asnumpy()]) ) , axis = 0)
  labels.append(dataset[item][1])
  if item%100 ==0 :
    print(item)

In [ ]:
print(str(int(labels[5999])))

34


new_data = {'ID': 6, 'label': 'F'}
df = df.append(new_data, ignore_index=True)

In [ ]:
# data = {
#     'ID' : [] ,
#     'Label' : []
# }
# label = pd.DataFrame(data)

In [ ]:
# for item in range(len(images)) :
#   new_data = {'ID' : str(int(item)),
#               'Label' : str(int(labels[item]))}
#   label = label.append(new_data , ignore_index = True)
#   cv2.imwrite("/content/drive/MyDrive/Project_Brain tumor/FaceNet/images/"+str(int(item)) +".jpeg" , images[item])

Streaming output truncated to the last 5000 lines.
<ipython-input-42-89cf5ed81903>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label = label.append(new_data , ignore_index = True)
<ipython-input-42-89cf5ed81903>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label = label.append(new_data , ignore_index = True)
<ipython-input-42-89cf5ed81903>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label = label.append(new_data , ignore_index = True)
<ipython-input-42-89cf5ed81903>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label = label.append(new_data , ignore_index = True)
<ipython-input-42-89cf5ed81903>:4: FutureWarning: The fra

In [ ]:
# label.to_csv("/content/drive/MyDrive/Project_Brain tumor/FaceNet/Metadata.csv")

In [ ]:
label = pd.read_csv("/content/drive/MyDrive/Project_Brain tumor/FaceNet/Metadata.csv")
label.pop("Unnamed: 0")

0          0
1          1
2          2
3          3
4          4
        ... 
5995    5995
5996    5996
5997    5997
5998    5998
5999    5999
Name: Unnamed: 0, Length: 6000, dtype: int64

In [ ]:
# shuffling the data befor dividing them into test and train
label = label.sample(frac = 1)

In [ ]:
data_train , data_test = train_test_split( label, test_size=0.2 , random_state=42)

In [ ]:
data_test = data_test.reset_index()
data_train = data_train.reset_index()
data_test.pop("index")
data_train.pop("index")


0       1582
1       1810
2       3124
3        632
4       3864
        ... 
4795    3116
4796    2912
4797    4389
4798    1643
4799    3142
Name: index, Length: 4800, dtype: int64

In [ ]:
data_train.to_csv("/content/drive/MyDrive/Project_Brain tumor/FaceNet/train.csv")
data_test.to_csv("/content/drive/MyDrive/Project_Brain tumor/FaceNet/test.csv")

In [ ]:
print(data_train)

        ID  Label
0     1582     12
1     1810     14
2     3124     21
3      632      3
4     3864     23
...    ...    ...
4795  3116     21
4796  2912     20
4797  4389     25
4798  1643     12
4799  3142     21

[4800 rows x 2 columns]


In [ ]:
trin_path = "/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset/Train"
test_path = "/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset/Test"

In [ ]:
import shutil
# moving dat to considered directory
for item in range(len(data_train)) :
  shutil.copy("/content/drive/MyDrive/Project_Brain tumor/FaceNet/images/"+str(data_train["ID"][item])+".jpeg", "/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset/Train/"+str(data_train['ID'][item]) + ".jpeg")


In [ ]:
import shutil
# moving test data to considered directory
for item in range(len(data_test)) :
  shutil.copy("/content/drive/MyDrive/Project_Brain tumor/FaceNet/images/"+str(data_test["ID"][item])+".jpeg", "/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset/Test/"+str(data_test['ID'][item]) + ".jpeg")


In [ ]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()

In [9]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from PIL import Image
import random

transform = transforms.Compose([
    transforms.Resize((299, 299)),  # InceptionResNetV2 expects input size (299, 299)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])




#  Coustom dataset
class data_Dataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def get_Anchor(self, anchor) :

      positive_list = self.img_labels[self.img_labels["Label"] == anchor]
      negative_list = self.img_labels[self.img_labels["Label"] != anchor]

      positive = positive_list.sample()
      negative = negative_list.sample()

      positive_img_path = self.img_dir+"/"+ str(positive.iloc[0,1])+".jpeg"
      Pos_mage = Image.open(positive_img_path)

      negative_img_path = self.img_dir+"/"+ str(negative.iloc[0,1])+".jpeg"
      Neg_mage = Image.open(negative_img_path)

      if self.transform:
          Posimage = self.transform(Pos_mage)
          Negimage = self.transform(Neg_mage)


      return Posimage , Negimage


    def __getitem__(self, idx):
        img_path = self.img_dir+"/"+ str(self.img_labels['ID'][idx])+".jpeg"
        image = Image.open(img_path)
        label = torch.tensor(self.img_labels["Label"][idx])
        if self.transform:
          image = self.transform(image)
        Positive  ,  Negative  = self.get_Anchor(int(label))

        return   image , Positive , Negative , label


train_data = data_Dataset("/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset/train.csv" , "/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset/Train" , transform)
test_data  = data_Dataset("/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset/test.csv" , "/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset/Test" , transform)

In [10]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=True)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F




class SoftAttention(nn.Module):


    def __init__(self, ch, m, concat_with_x=False, aggregate=False):
        super(SoftAttention, self).__init__()

        self.channels = int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x

        kernel_shape_conv3d = (self.multiheads ,  self.channels ,3, 3 , 1 )
        self.kernel_conv3d = nn.Parameter(torch.Tensor(*kernel_shape_conv3d))
        nn.init.kaiming_uniform_(self.kernel_conv3d, a=0, mode='fan_in', nonlinearity='relu')

        self.bias_conv3d = nn.Parameter(torch.zeros(self.multiheads))

    def forward(self, x):
        exp_x = x.unsqueeze(-1)
        c3d = F.conv3d(exp_x, self.kernel_conv3d, stride=(1, 1, x.shape[-1]), padding=(1, 1, 0))

        conv3d = c3d + self.bias_conv3d.view(1, -1, 1, 1, 1)
        conv3d = F.relu(conv3d)

        conv3d = conv3d.permute(0, 4, 1, 2, 3)
        conv3d = conv3d.squeeze(-1)
        conv3d = conv3d.view(-1, self.multiheads, 100)

        softmax_alpha = F.softmax(conv3d, dim=-1)
        softmax_alpha = softmax_alpha.view(-1, self.multiheads, 10 , 10)

        if self.aggregate_channels == True:
            exp_softmax_alpha = softmax_alpha.unsqueeze(-1)
            exp_softmax_alpha = exp_softmax_alpha.permute(0, 2, 3, 1, 4)

            x_exp = x.unsqueeze(-2)

            u = exp_softmax_alpha * x_exp
            u = u.view(-1, self.i_shape[1], self.i_shape[2], u.shape[-1] * u.shape[-2])

        else:
            exp_softmax_alpha = softmax_alpha.permute(0, 2, 3, 1)
            exp_softmax_alpha = exp_softmax_alpha.sum(dim=-1).unsqueeze(0)
            exp_softmax_alpha = exp_softmax_alpha.permute(1 ,  0 , 2 , 3)


            u = exp_softmax_alpha * x
        if self.concat_input_with_scaled:
            o = torch.cat([u, x], dim=-1)
        else:
            o = u

        return [o, softmax_alpha]

    def compute_output_shape(self, input_shape):
        # Assuming input_shape is a tuple of dimensions (batch_size, channels, height, width)
        out_attention_maps_shape = (input_shape[0], self.multiheads, input_shape[2], input_shape[3])

        if self.aggregate_channels == False:
            out_features_shape = (input_shape[0], input_shape[1] + input_shape[1] * self.multiheads)
        else:
            out_features_shape = (input_shape[0], input_shape[1] * 2) if self.concat_input_with_scaled else input_shape

        return [out_features_shape, out_attention_maps_shape]


In [5]:
# model
from torchvision import datasets, models, transforms
import torch.nn.functional as F
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class final_model(nn.Module):

    def __init__(self  ) :

      super(final_model , self).__init__()
      self.model1 = models.resnet50(pretrained='vggface2')

      for i in self.model1.parameters():
        i.requires_grad = False
      for i in self.model1.avgpool.parameters() :
        i.requires_grad = True
      for i in self.model1.layer4.parameters():
          i.requires_grad = True



      self.embedding = torch.nn.Linear(1000,128)


    def forward(self, x):
        y = self.model1(x)
        y1 = F.normalize(y, p=2, dim=1)






        result = self.embedding(y1)
        return result

main_model = final_model().to(device)


Using cuda device


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 92.4MB/s]


In [ ]:
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import torch.nn as nn
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class final_model_with_soft_attention(nn.Module):

    def __init__(self):
              super(final_model_with_soft_attention , self).__init__()
              self.model1 = models.resnet50(pretrained='vggface2')

              for i in self.model1.parameters():
                i.requires_grad = False

              for i in self.model1.layer4.parameters():
                i.requires_grad = True

              self.model1 = torch.nn.Sequential(*list(self.model1.children())[:-2])
              self.soft_attention_layer = SoftAttention(ch=2048, m=16)
              self.max_pooling = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))


              self.embedding = torch.nn.Linear(102400,128)

    def forward(self, x ) :
      conv = self.model1(x)
#       last_layer = self.model1[-1]
#       self.channels = last_layer.out_channels if hasattr(last_layer, 'out_channels') else last_layer[-1].out_channels
# # Instantiate SoftAttention layer

      # Apply Soft Attention
      attention_layer, attention_maps =  self.soft_attention_layer(conv)

      # Apply max pooling to both attention_layer and conv
      attention_layer = self.max_pooling(attention_layer)
      conv = self.max_pooling(conv)

      # Concatenate tensors along the channel dimension
      concatenated = torch.cat([conv, attention_layer], dim=1)

      # Apply activation, dropout, etc.
      conv = F.relu(concatenated)
      conv = F.dropout(conv, p=0.5, training=self.training)

      # Flatten and normalize
      y = torch.flatten(conv, 1)

      y1 = F.normalize(y, p=2, dim=1)
      # Apply linear embedding layer
      result = self.embedding(y1)
      return result

model_soft_attention = final_model_with_soft_attention().to(device)


Using cuda device


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 164MB/s]


In [ ]:
triplet_loss = nn.TripletMarginLoss(margin=0.1, p=2, eps=1e-7)
optimizer = torch.optim.Adam(model_soft_attention.parameters(), lr=0.00001)


epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_soft_attention , triplet_loss, optimizer)
    test(test_dataloader, model_soft_attention, triplet_loss)
print("Done!")

Epoch 1
-------------------------------
Avg loss: 0.099775 

Avg loss: 0.099663 

Epoch 2
-------------------------------
Avg loss: 0.099411 

Avg loss: 0.098950 

Epoch 3
-------------------------------
Avg loss: 0.097605 

Avg loss: 0.094324 

Epoch 4
-------------------------------
Avg loss: 0.076907 

Avg loss: 0.066586 

Epoch 5
-------------------------------
Avg loss: 0.058130 

Avg loss: 0.062074 

Epoch 6
-------------------------------
Avg loss: 0.053370 

Avg loss: 0.058238 

Epoch 7
-------------------------------
Avg loss: 0.050365 

Avg loss: 0.061049 

Epoch 8
-------------------------------
Avg loss: 0.047740 

Avg loss: 0.058488 

Epoch 9
-------------------------------
Avg loss: 0.044873 

Avg loss: 0.055626 

Epoch 10
-------------------------------
Avg loss: 0.042435 

Avg loss: 0.055747 

Epoch 11
-------------------------------
Avg loss: 0.042120 

Avg loss: 0.060177 

Epoch 12
-------------------------------
Avg loss: 0.042492 

Avg loss: 0.057596 

Epoch 13
----

In [11]:
# loss_fn = TripletLoss()
triplet_loss = nn.TripletMarginLoss(margin=0.1, p=2, eps=1e-7)
optimizer = torch.optim.Adam(main_model.parameters(), lr=0.00001)


epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, main_model , triplet_loss, optimizer)
    test(test_dataloader, main_model,triplet_loss )

print("Done!")

Epoch 1
-------------------------------
Avg loss: 0.089513 

Avg loss: 0.084997 

Epoch 2
-------------------------------
Avg loss: 0.077900 

Avg loss: 0.073497 

Epoch 3
-------------------------------
Avg loss: 0.066462 

Avg loss: 0.065809 

Epoch 4
-------------------------------
Avg loss: 0.056739 

Avg loss: 0.058979 

Epoch 5
-------------------------------
Avg loss: 0.051761 

Avg loss: 0.054212 

Epoch 6
-------------------------------
Avg loss: 0.045864 

Avg loss: 0.057074 

Epoch 7
-------------------------------
Avg loss: 0.042250 

Avg loss: 0.051945 

Epoch 8
-------------------------------
Avg loss: 0.040063 

Avg loss: 0.052733 

Epoch 9
-------------------------------
Avg loss: 0.039767 

Avg loss: 0.049668 

Epoch 10
-------------------------------
Avg loss: 0.035829 

Avg loss: 0.049759 

Epoch 11
-------------------------------
Avg loss: 0.032179 

Avg loss: 0.049122 

Epoch 12
-------------------------------
Avg loss: 0.031307 

Avg loss: 0.046293 

Epoch 13
----

In [ ]:
test( test_dataloader, main_model , triplet_loss)


Avg loss: 0.136466 



In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = dataloader.__len__()
    data_worker = data_Dataset("/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset/train.csv" , "/content/drive/MyDrive/Project_Brain tumor/FaceNet/Dataset/Train" , transform)

    model.train()
    total_loss = 0
    num_batchs = 0
    for batch, (X , P , N ,  y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        P = P.to(device)
        N = N.to(device)

        # Compute prediction error
        anchor_out = model(X)




        with torch.no_grad() :
           positive_out = model(P)
           negative_out = model(N)

        loss = loss_fn(anchor_out, positive_out , negative_out )
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # if batch % 100 == 0:
        loss, current = loss.item(), (batch + 1) * len(X)
        total_loss += loss
        num_batchs += 1
        # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    total_loss /= num_batchs
    print(f"Avg loss: {total_loss:>8f} \n")

In [6]:
def test(dataloader, model, loss_fn):
    size = dataloader.__len__()

    total_loss = 0
    num_batchs = 0
    for batch, (X , P , N ,  y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        P = P.to(device)
        N = N.to(device)

        # Compute prediction error




        with torch.no_grad() :
           anchor_out = model(X)
           positive_out = model(P)
           negative_out = model(N)

        loss = loss_fn(anchor_out, positive_out , negative_out )
        loss, current = loss.item(), (batch + 1) * len(X)
        total_loss += loss
        num_batchs += 1
    total_loss /= num_batchs
    print(f"Avg loss: {total_loss:>8f} \n")